
# CP322 Machine Learning — Assignment 1 (Completed Code)

**Name:** _Your Name Here_  
**Course:** CP322 — Fall 2025  
**Dataset:** NYC Airbnb Open Data (price as target)

> Set `CSV_PATH` below to your dataset file (e.g., `AB_NYC_2019.csv`) and run the notebook top-to-bottom.


## 0) Setup

In [ ]:

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ML tools
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

# Mapping
import folium
from folium.plugins import HeatMap

# Utility
from IPython.display import display

pd.set_option('display.max_columns', 200)
plt.rcParams['figure.figsize'] = (8, 5)

def say(msg: str):
    print(f"✨ {msg}")

print("✅ Libraries loaded.")


## 0.1) Load Data

In [ ]:

# TODO: Set this to your CSV path
CSV_PATH = "AB_NYC_2019.csv"  # <-- change to the actual path

df = pd.read_csv(CSV_PATH)
say(f"Loaded: {df.shape[0]} rows × {df.shape[1]} columns")
display(df.head())


## 1) Exploratory Data Analysis (3 pts)

### 1.1 Descriptive statistics

In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
desc = df[num_cols].describe().T
display(desc)


### 1.2 Handle missing values

In [ ]:

# Drop rows where reviews_per_month is missing
if 'reviews_per_month' in df.columns:
    before = len(df)
    df = df.dropna(subset=['reviews_per_month'])
    say(f"Dropped {before - len(df)} rows with missing reviews_per_month.")
else:
    say("No 'reviews_per_month' column found; skipping drop.")

# Replace missing dates in last_review with 'Unknown'
if 'last_review' in df.columns:
    df['last_review'] = df['last_review'].astype('string').fillna('Unknown')
    say("Filled missing 'last_review' with 'Unknown' and coerced to string.")
else:
    say("No 'last_review' column found; skipping fill.")

# Median for numerics / Mode for categoricals
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

num_filled = 0
for c in numeric_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())
        num_filled += 1

cat_filled = 0
for c in categorical_cols:
    if df[c].isna().any():
        mode = df[c].mode(dropna=True)
        fill_val = mode.iloc[0] if len(mode) else "Unknown"
        df[c] = df[c].fillna(fill_val)
        cat_filled += 1

say(f"Imputed {num_filled} numeric and {cat_filled} categorical columns.")


### 1.3 Detect & remove price outliers with IQR

In [ ]:

assert 'price' in df.columns, "Expected a 'price' column in dataset."
Q1, Q3 = df['price'].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
before = len(df)
df = df[(df['price'] >= lower) & (df['price'] <= upper)]
say(f"IQR filter on price: bounds [{lower:.2f}, {upper:.2f}]. Removed {before - len(df)} rows.")


### 1.4 Visualizations

In [ ]:

# Histogram of price
ax = df['price'].plot(kind='hist', bins=50, title='Histogram of Price')
ax.set_xlabel('Price')
plt.show()

# Scatter: price vs number_of_reviews
if 'number_of_reviews' in df.columns:
    plt.scatter(df['number_of_reviews'], df['price'], alpha=0.3)
    plt.title('Price vs Number of Reviews')
    plt.xlabel('Number of Reviews')
    plt.ylabel('Price')
    plt.show()

# Boxplots: price by room_type
if 'room_type' in df.columns:
    sns.boxplot(data=df, x='room_type', y='price')
    plt.title('Price by Room Type')
    plt.xticks(rotation=15)
    plt.show()


### 1.5 Correlation heatmap + brief discussion

In [ ]:

corr_features = ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
                 'calculated_host_listings_count', 'availability_365']
existing = [c for c in corr_features if c in df.columns]
corr = df[existing].corr(numeric_only=True)

sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Heatmap (Selected Features)")
plt.show()

print("Notes: discuss any strong/weak correlations and potential multicollinearity.")


## 2) Feature Engineering (3 pts)

In [ ]:

# One-hot encode neighbourhood_group and room_type later via ColumnTransformer
cat_cols_needed = [col for col in ['neighbourhood_group', 'room_type'] if col in df.columns]

# Derived feature: price_per_accommodates (price / minimum_nights)
if 'price' in df.columns and 'minimum_nights' in df.columns:
    df['price_per_accommodates'] = df['price'] / df['minimum_nights'].replace(0, np.nan)
    df['price_per_accommodates'] = df['price_per_accommodates'].fillna(0)
    say("Created derived feature 'price_per_accommodates'.")
else:
    say("Missing 'price' or 'minimum_nights'; skipping derived feature.")

num_for_scaler = [c for c in [
    'minimum_nights', 'number_of_reviews', 'reviews_per_month',
    'calculated_host_listings_count', 'availability_365', 'price_per_accommodates'
] if c in df.columns]

say(f"Categoricals to encode later: {cat_cols_needed}")
say(f"Numerics to scale later: {num_for_scaler}")


## 3) Baseline Regression Models (2 pts)

In [ ]:

feature_cols = [c for c in [
    'neighbourhood_group', 'room_type',
    'minimum_nights', 'number_of_reviews', 'reviews_per_month',
    'calculated_host_listings_count', 'availability_365'
] if c in df.columns]

target_col = 'price'
assert target_col in df.columns, "Missing target 'price'."

X = df[feature_cols].copy()
y = df[target_col].copy()

cat_cols = [c for c in feature_cols if X[c].dtype == 'object' or str(X[c].dtype).startswith('string')]
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('num', StandardScaler(), num_cols)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linreg = Pipeline(steps=[
    ('prep', preprocess),
    ('model', LinearRegression())
])

linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred, squared=True)
r2 = r2_score(y_test, y_pred)

say(f"Baseline Linear Regression — RMSE: {rmse:.2f}, MSE: {mse:.2f}, R²: {r2:.3f}")


## 4) Lasso, Ridge, and ElasticNet Regression (2 pts)

In [ ]:

alphas = [0.01, 0.1, 1, 10, 100]

def eval_model(pipeline, name):
    pipeline.fit(X_train, y_train)
    pred = pipeline.predict(X_test)
    rmse = mean_squared_error(y_test, pred, squared=False)
    mse = mean_squared_error(y_test, pred, squared=True)
    r2 = r2_score(y_test, pred)
    say(f"{name} → RMSE: {rmse:.2f}, MSE: {mse:.2f}, R²: {r2:.3f}")
    return {'model': name, 'rmse': rmse, 'mse': mse, 'r2': r2}

# Lasso
lasso = Pipeline([('prep', preprocess), ('model', Lasso(max_iter=10000))])
gs_lasso = GridSearchCV(lasso, param_grid={'model__alpha': alphas}, cv=5, n_jobs=-1)
res_lasso = eval_model(gs_lasso, "Lasso")

# Ridge
ridge = Pipeline([('prep', preprocess), ('model', Ridge())])
gs_ridge = GridSearchCV(ridge, param_grid={'model__alpha': alphas}, cv=5, n_jobs=-1)
res_ridge = eval_model(gs_ridge, "Ridge")

# ElasticNet
enet = Pipeline([('prep', preprocess), ('model', ElasticNet(max_iter=10000))])
enet_grid = {'model__alpha': alphas, 'model__l1_ratio': [0.1, 0.5, 0.9]}
gs_enet = GridSearchCV(enet, param_grid=enet_grid, cv=5, n_jobs=-1)
res_enet = eval_model(gs_enet, "ElasticNet")

results_df = pd.DataFrame([res_lasso, res_ridge, res_enet]).sort_values('rmse')
display(results_df.reset_index(drop=True))


## 5) Bias–Variance Tradeoff & Model Complexity (3 pts)

In [ ]:

# Prepare lists again (from baseline step)
cat_cols = [c for c in feature_cols if X[c].dtype == 'object' or str(X[c].dtype).startswith('string')]
num_cols = [c for c in feature_cols if c not in cat_cols]
assert len(num_cols) > 0, "Need numeric predictors for polynomial features."

def poly_pipeline(deg):
    num_pipe = Pipeline([('poly', PolynomialFeatures(degree=deg, include_bias=False)),
                         ('sc', StandardScaler())])
    poly_ct = ColumnTransformer([
        ('num', num_pipe, num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ])
    return Pipeline([('prep', poly_ct), ('model', LinearRegression())])

degrees = [1, 2, 3, 4]
poly_scores = []

for d in degrees:
    pipe = poly_pipeline(d)
    train_sizes, train_scores, val_scores = learning_curve(
        estimator=pipe,
        X=X, y=y,
        cv=5,
        train_sizes=np.linspace(0.2, 1.0, 5),
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    train_rmse = np.sqrt(-train_scores)
    val_rmse = np.sqrt(-val_scores)

    plt.figure()
    plt.plot(train_sizes, train_rmse.mean(axis=1), marker='o', label='Train RMSE')
    plt.plot(train_sizes, val_rmse.mean(axis=1), marker='s', label='Validation RMSE')
    plt.title(f'Learning Curve (Degree {d})')
    plt.xlabel('Training Samples')
    plt.ylabel('RMSE')
    plt.legend()
    plt.show()

    # Final fit for test performance
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    rmse = mean_squared_error(y_te, preds, squared=False)
    r2 = r2_score(y_te, preds)
    poly_scores.append({'degree': d, 'rmse': rmse, 'r2': r2})

poly_df = pd.DataFrame(poly_scores).sort_values('degree')
display(poly_df)



**Discussion (Bias–Variance):**  
- Comment on where the model underfits (high RMSE on train & val at low degree).  
- Comment on where the model overfits (train RMSE drops while val RMSE worsens at high degree).  
- Identify the degree with best validation/test RMSE and briefly justify.


## 6) Advanced Visualization (2 pts)

### 6.1 Geographic price heatmap

In [ ]:

if all(col in df.columns for col in ['latitude', 'longitude', 'price']):
    m = folium.Map(location=[40.7128, -74.0060], zoom_start=10, tiles='cartodbpositron')
    heat_data = df[['latitude', 'longitude', 'price']].dropna().values.tolist()
    HeatMap(heat_data, radius=8, blur=15, max_zoom=13).add_to(m)
    m
else:
    say("latitude/longitude/price missing; skipping heatmap.")


### 6.2 Predicted vs Actual for best model

In [ ]:

# Pick best regularized model by RMSE (from results_df)
best_name = results_df.iloc[0]['model']
say(f"Best model from Section 4: {best_name}")

best_estimator = {'Lasso': gs_lasso, 'Ridge': gs_ridge, 'ElasticNet': gs_enet}[best_name]
best_estimator.fit(X_train, y_train)
preds = best_estimator.predict(X_test)

plt.scatter(y_test, preds, alpha=0.4)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title(f'Predicted vs Actual — {best_name}')
lims = [min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])]
plt.plot(lims, lims, '--')
plt.xlim(lims); plt.ylim(lims)
plt.show()


## Appendix


- Assumptions and deviations, if any.  
- Data source and library versions.
